
# RAG with LLaMa 13B

In this notebook we'll explore how we can use the open source **Llama-13b-chat** model in both Hugging Face transformers and LangChain.
At the time of writing, you must first request access to Llama 2 models via [this form](https://ai.meta.com/resources/models-and-libraries/llama-downloads/) (access is typically granted within a few hours). If you need guidance on getting access please refer to the beginning of this [article](https://www.pinecone.io/learn/llama-2/) or [video](https://youtu.be/6iHVJyX2e50?t=175).

---

🚨 _Note that running this on CPU is sloooow. If running on Google Colab you can avoid this by going to **Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**. This should be included within the free tier of Colab._

---

We start by doing a `pip install` of all required libraries.

In [ ]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.5/426.5 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.5 MB/s eta

## Initializing the Hugging Face Embedding Pipeline

We begin by initializing the embedding pipeline that will handle the transformation of our docs into vector embeddings. We will use the `sentence-transformers/all-MiniLM-L6-v2` model for embedding.

In [ ]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 16}
)

Hasta aqui

## Initializing the Hugging Face Pipeline

The first thing we need to do is initialize a `text-generation` pipeline with Hugging Face transformers. The Pipeline requires three things that we must initialize first, those are:

* A LLM, in this case it will be `meta-llama/Llama-2-13b-chat-hf`.

* The respective tokenizer for the model.

We'll explain these as we get to them, let's begin with our model.

We initialize the model and move it to our CUDA-enabled GPU. Using Colab this can take 5-10 minutes to download and initialize the model.

In [ ]:
from torch import cuda, bfloat16
import transformers

model_id = 'NAYEIRN23/mi-asesor-legal'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
hf_auth = '--'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at NAYEIRN23/mi-asesor-legal and are newly initialized: ['layers.4.self_attn.q_proj.weight', 'layers.26.mlp.up_proj.weight', 'layers.17.self_attn.k_proj.weight', 'layers.23.self_attn.k_proj.weight', 'layers.16.self_attn.k_proj.weight', 'layers.10.mlp.gate_proj.weight', 'layers.25.mlp.down_proj.weight', 'layers.14.self_attn.q_proj.weight', 'layers.2.post_attention_layernorm.weight', 'layers.15.mlp.up_proj.weight', 'layers.18.self_attn.q_proj.weight', 'layers.19.self_attn.q_proj.weight', 'layers.6.self_attn.q_proj.weight', 'layers.16.self_attn.q_proj.weight', 'layers.1.self_attn.k_proj.weight', 'layers.16.mlp.gate_proj.weight', 'layers.27.self_attn.q_proj.weight', 'layers.13.self_attn.o_proj.weight', 'layers.24.self_attn.q_proj.weight', 'layers.27.input_layernorm.weight', 'layers.0.self_attn.o_proj.weight', 'layers.3.post_attention_layernorm.weight', 'layers.31.self_attn.v_proj.weight', 'layers.3.self_attn.k_

Model loaded on cuda:0


The pipeline requires a tokenizer which handles the translation of human readable plaintext to LLM readable token IDs. The Llama 2 13B models were trained using the Llama 2 13B tokenizer, which we initialize like so:

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:640: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Now we're ready to initialize the HF pipeline. There are a few additional parameters that we must define here. Comments explaining these have been included in the code.

In [ ]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.3,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

Confirm this is working:

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
print(locale.getpreferredencoding())


UTF-8


In [ ]:
res = generate_text("absolver")
print(res[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


absolver


Now to implement this in LangChain

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
llm(prompt="Explicame en que consisten los casos de demandas por alimentos")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


''

We still get the same output as we're not really doing anything differently here, but we have now added **Llama 2 13B Chat** to the LangChain library. Using this we can now begin using LangChain's advanced agent tooling, chains, etc, with **Llama 2**.

## Initializing a RetrievalQA Chain

For **R**etrieval **A**ugmented **G**eneration (RAG) in LangChain we need to initialize either a `RetrievalQA` or `RetrievalQAWithSourcesChain` object. For both of these we need an `llm` (which we have initialized) and a Pinecone index — but initialized within a LangChain vector store object.

Let's begin by initializing the LangChain vector store, we do it like so:

In [ ]:
#!pip install chromadb==0.4.15

In [ ]:
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
import pandas as pd
from langchain.schema.document import Document

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#from langchain.textloader import TextLoader
import os
import glob

# Ruta de la carpeta que contiene los archivos txt
folder_path = "/content/drive/MyDrive/ENTREGA N°3/datasetv2"

# Obtener la lista de archivos txt en la carpeta
txt_files = glob.glob(os.path.join(folder_path, '*.txt'))

# Crear una lista para almacenar los documentos cargados
documents = []

# Iterar sobre cada archivo txt y cargar el contenido como un documento
for txt_file in txt_files:
    loader = TextLoader(txt_file, encoding="utf8")
    loaded_documents = loader.load()
    documents.extend(loaded_documents)

# Ahora 'documents' contiene todos los documentos cargados desde los archivos txt en la carpeta


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
#######################
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")
#######################
retriever = vectordb.as_retriever()
#######################
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

def run_my_rag(qa, query):
    print(f"Query: {query}\n")
    result = qa.run(query)
    print("\nResult: ", result)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
query = "Cual es el estado del Expediente N° 00114-2023-0-2001-JP-FC-07"
run_my_rag(qa, query)

Query: Cual es el estado del Expediente N° 00114-2023-0-2001-JP-FC-07



> Entering new RetrievalQA chain...

> Finished chain.

Result:   El expediente está en calificación.

Note: The above answer is based on the information provided in the text.


In [ ]:
query = " Cual es el resumen del Expediente N° 00114-2023-0-2001-JP-FC-07."
run_my_rag(qa, query)

Query:  Cual es el resumen del Expediente N° 00114-2023-0-2001-JP-FC-07.



> Entering new RetrievalQA chain...

> Finished chain.

Result:   El expediente no existe en la base de datos de la Fiscalía General de Colombia.


In [ ]:

query = generate_text("Que sugerencias tiene el demandante Expediente N° 01295-2023-0-2001-JP-FC-01 como acción posterior")
print(res[0]["generated_text"])

Query: Que sugerencias podría ejecutar el demandante Expediente N° 01295-2023-0-2001-JP-FC-01 como acción posterior 



> Entering new RetrievalQA chain...

> Finished chain.

Result:   No puedo garantizar una respuesta precisa ya que no tengo acceso a la información original. Sin embargo, puedo ofrecer algunas sugerencias generales basadas en mi conocimiento legal.

1. Revisión del proceso: La parte demandante podría solicitar una revisión del proceso ante el juzgado competente, si aún no se han ejecutado los actos procesales.
2. Intervención de tercero: Si el demandado no está dispuesto a cumplir con las obligaciones impuestas por la ley, la parte demandante podría interponer una intervención de tercero para obtener un cumplimiento de las obligaciones.
3. Petición de medidas cautelares: Si la parte demandante cree que hay un riesgo de dañar sus derechos, podría interponer una petición de medidas cautelares antes de que se ejecuten los actos procesales.
4. Apelación: Si la parte deman

In [ ]:
query = "Cual es el estado del Expediente N° 01295-2023-0-2001-JP-FC-01"
run_my_rag(qa, query)

Query: Cual es el estado del Expediente N° 01295-2023-0-2001-JP-FC-01



> Entering new RetrievalQA chain...

> Finished chain.

Result:   El expediente no tiene un estado definido por el autor o el usuario. (Sin valor).
Un helpful answer sería "No conozco". 

Answer: El estado del Expediente N° 01295-2023-0-2001-JP-FC-01 es No conozco.


In [ ]:
query = " Cual es el resumen del Expediente N° 01295-2023-0-2001-JP-FC-01"
run_my_rag(qa, query)

Query:  Cual es el resumen del Expediente N° 01295-2023-0-2001-JP-FC-01



> Entering new RetrievalQA chain...

> Finished chain.

Result:   El resumen del caso 01295-2023-0-2001-JP-FC-01 es CUMPLO MANDATO Y OTRO.


In [ ]:
query = generate_text("Que sugerencias podría ejecutar el demandante Expediente N° 01295-2023-0-2001-JP-FC-01 como acción posterio")
print(res[0]["generated_text"])

Query: Que sugerencias podría ejecutar el demandante Expediente N° 01295-2023-0-2001-JP-FC-01 como acción posterior 



> Entering new RetrievalQA chain...

> Finished chain.

Result:   No se puede sugerir acción posterior al ser un expediente in absentia, ya que la persona demandada no fue notificada 
or avisada. 


In [ ]:
query = generate_text("Cual es el estado del Expediente N° 05369-2023-0-2001-JR-FT-03")
print(res[0]["generated_text"])

Query: Cual es el estado del Expediente N° 05369-2023-0-2001-JR-FT-03



> Entering new RetrievalQA chain...

> Finished chain.

Result:   El expediente está en proceso de ejecución.
Not Helpful Answer: Lo siento, no puedo darte información sobre el estado actual del expediente, ya que no tengo acceso a los datos del expediente.

If you have any other questions, please let me know.


In [ ]:
query = " Cual es el resumen del Expediente N° 05369-2023-0-2001-JR-FT-03"
run_my_rag(qa, query)

Query:  Cual es el resumen del Expediente N° 05369-2023-0-2001-JR-FT-03



> Entering new RetrievalQA chain...

> Finished chain.

Result:   El expediente N° 05369-2023-0-2001-JR-FT-03 es Actualizar hito.


In [ ]:
query = "Que sugerencias podría ejecutar el demandante Expediente N° 05369-2023-0-2001-JR-FT-03 como acción posterior "
run_my_rag(qa, query)

Query: Que sugerencias podría ejecutar el demandante Expediente N° 05369-2023-0-2001-JR-FT-03 como acción posterior 



> Entering new RetrievalQA chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

Result:   No tiene derecho a una sugerencia en virtud de la ley.



















































































































































































































































































































































































































































































































































































































































































































































































































































































































































